<div class="alert-danger">
    Faire une liste des installs a faire from scratch pour runner le notebook avec jupyter
    
</div>

# Tutorial Notebook for IBoat - PMCTS

<div class="alert-danger">
*Please be aware that this tutorials shall not replace the project thorought documentation that you can find at :
https://pbarde.github.io/IBoatPIE/ * 
</div>

<div class="alert-success"> After completing this tutorial you should be able to use all the tools devellopped during the IBoat - PMCTS projet. Using these tools you should investigate different algorithms tunings to find an optimal strategy and compare it to the isochrones method.
</div>

## Weather forecasts and Simulators

In this section you'll learn how to download, load, process and visualize weather forecasts. We will also create the simulators that are used in the PMCTS.

### Downloading weather forecasts

Let's start by downloading some forecasts. 
<div class="alert-warning">
*You'll need to change the* `mydate` *variable* </div>

In [1]:
import forest as ft

# The starting day of the forecast. If it's too ancient, the forecast might not be available anymore
mydate = '20180228' # for February 2, 2018

# We will download the mean scenario (id=0) and the first 2 perturbed scenarios
scenario_ids = range(3)
ft.download_scenarios(mydate, latBound=[40, 50], lonBound=[-15 + 360, 360], scenario_ids=scenario_ids)

Saved into : ../data/20180228_0000z.obj
Saved into : ../data/20180228_0100z.obj
Saved into : ../data/20180228_0200z.obj


### Loading weather objects


Now that we have downaloaded some forecasts, let's load and create simulators out of them.  

In [1]:
import forest as ft

# The starting day of the forecast. If it's too ancient, the forecast might not be available anymore
mydate = '20180228' # for February 2, 2018

# We will download the mean scenario (id=0) and the first 2 perturbed scenarios
scenario_ids = range(3)
Weathers = ft.load_scenarios(mydate, latBound=[40, 50], lonBound=[-15 + 360, 360], scenario_ids=scenario_ids)

Loaded : ../data/20180228_0000z.obj
Loaded : ../data/20180228_0100z.obj
Loaded : ../data/20180228_0200z.obj


### Create simulators and displaying wind conditions

We define the main parameters of our simulators, we create them and we visualize the corresponding wind conditions. The plots are interactives (use the upper-left icons to navigate through the weather forecast).
<div class="alert-warning">
*Please don't mind the long output as it is a side effect of the basemap library we are using (double click on the left side of it to hide it).*<br />
*Moreover, our code is not supposed to be executed in a jupyter notebook which is a bit capricious and does not handle mutli-processing properly. To visualize multiple scenarios interactively from a script you should use* `ft.play_multiple_scenarios(Sims)`
</div>

In [2]:
%%capture  
% matplotlib tk
from time import sleep
NUMBER_OF_SIM = 3  # <=20 
SIM_TIME_STEP = 6  # in hours
STATE_INIT = [0, 44, 355]
N_DAYS_SIM = 3  # time horizon in days

Sims = ft.create_simulators(Weathers, numberofsim=NUMBER_OF_SIM, simtimestep=SIM_TIME_STEP,
                            stateinit=STATE_INIT, ndaysim=N_DAYS_SIM)

## for interactive display of scenarios inside a notebook
#for ii, sim in enumerate(Sims) : 
#    sim.play_scenario(ii)
#    sleep(1)
    
## /!\ if executing from a .py script, you better use as it uses multiprocessing: 
# ft.play_multiple_scenarios(Sims)

## The Parallel Monte-Carlo Tree Search

In this section we will see how to launch a PMCTS and visualize the results.

### Initialisation of the search

First of all we define a departure point, a mission heading and we compute the corresponding destination point and reference travel time. And we visualize the mean trajectories per scenarios during the two initialization phases. 

In [4]:
%%capture
import matplotlib
matplotlib.rcParams.update({'font.size': 10})
missionheading = 0 # direction wrt. true North we want to go the furthest.
ntra = 50 # number of trajectories used during the initialization
destination, timemin = ft.initialize_simulators(Sims, ntra, STATE_INIT, missionheading, plot=True)

In [41]:
print("destination : " + str(destination) + "  &  timemin : " + str(timemin) + "\n")

destination : [48.60778768092585, 355.0]  &  timemin : 3.13841137437

